#### Calculate SenMayo and R-HSA-2559583 score for genes

SenMayo paper claims that their gene set is more robust for detecting senescent cells than R-HSA-2559583. We will calculate senescence scores for each individual cell type. We will then pseudobulk to get the mean senescence score at the donor level and perform covariate modeling to account for batch effects.

In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
import statsmodels.stats.multitest as smm
from statsmodels.stats.multitest import multipletests
import statsmodels.formula.api as smf

In [2]:
# open file with SenMayo gene set
with open('SenMayo_list.txt', 'r') as file:
    genes_string = file.read().strip()

# split the string into a list for scanpy
genes_list = genes_string.split(',')
SenMayo_list = genes_list

print(SenMayo_list)
print(len(SenMayo_list))

['ACVR1B', 'ANG', 'ANGPT1', 'ANGPTL4', 'AREG', 'AXL', 'BEX3', 'BMP2', 'BMP6', 'C3', 'CCL1', 'CCL13', 'CCL16', 'CCL2', 'CCL20', 'CCL24', 'CCL26', 'CCL3', 'CCL3L1', 'CCL4', 'CCL5', 'CCL7', 'CCL8', 'CD55', 'CD9', 'CSF1', 'CSF2', 'CSF2RB', 'CST4', 'CTNNB1', 'CTSB', 'CXCL1', 'CXCL10', 'CXCL12', 'CXCL16', 'CXCL2', 'CXCL3', 'CXCL8', 'CXCR2', 'DKK1', 'EDN1', 'EGF', 'EGFR', 'EREG', 'ESM1', 'ETS2', 'FAS', 'FGF1', 'FGF2', 'FGF7', 'GDF15', 'GEM', 'GMFG', 'HGF', 'HMGB1', 'ICAM1', 'ICAM3', 'IGF1', 'IGFBP1', 'IGFBP2', 'IGFBP3', 'IGFBP4', 'IGFBP5', 'IGFBP6', 'IGFBP7', 'IL10', 'IL13', 'IL15', 'IL18', 'IL1A', 'IL1B', 'IL2', 'IL32', 'IL6', 'IL6ST', 'IL7', 'INHA', 'IQGAP2', 'ITGA2', 'ITPKA', 'JUN', 'KITLG', 'LCP1', 'MIF', 'MMP1', 'MMP10', 'MMP12', 'MMP13', 'MMP14', 'MMP2', 'MMP3', 'MMP9', 'NAP1L4', 'NRG1', 'PAPPA', 'PECAM1', 'PGF', 'PIGF', 'PLAT', 'PLAU', 'PLAUR', 'PTBP1', 'PTGER2', 'PTGES', 'RPS6KA5', 'SCAMP4', 'SELPLG', 'SEMA3F', 'SERPINB4', 'SERPINE1', 'SERPINE2', 'SPP1', 'SPX', 'TIMP2', 'TNF', 'TNFRSF

In [3]:
# load in the reactome senescence gene set
reactome_senescence_list = list(pd.read_csv("R-HSA-2559583_list.txt", index_col = 0)['symbol'])
print(reactome_senescence_list)

['NBN', 'AGO1', 'H2BFS', 'RAD50', 'HIST2H2AC', 'TNRC6A', 'HIST1H2AJ', 'FOS', 'HIST1H2BJ', 'ANAPC7', 'IL1A', 'HMGA2', 'HIST1H2BK', 'RBBP4', 'PHC3', 'H2AFX', 'CDKN2A', 'ANAPC4', 'EHMT2', 'MIR24-2', 'EZH2', 'JUN', 'RBBP7', 'HIST1H1E', 'ACD', 'HIST1H2BC', 'MAPKAPK2', 'TERF1', 'TNIK', 'HIST1H1A', 'ANAPC2', 'CDKN1A', 'RPS6KA1', 'HIST2H3A', 'CCNA2', 'MAP2K4', 'CEBPB', 'UBE2C', 'FZR1', 'SP1', 'HIST1H2BA', 'MAPK3', 'MINK1', 'RING1', 'RELA', 'MAPK1', 'IGFBP7', 'HIST1H2BB', 'EP400', 'H3F3A', 'PHC2', 'CDC23', 'CXCL8', 'TNRC6B', 'MAPK14', 'MAP4K4', 'NFKB1', 'ERF', 'UBE2D1', 'MRE11A', 'HIST1H1C', 'CDC27', 'HIST2H2AA3', 'UBB', 'MAP2K3', 'H2AFZ', 'ETS1', 'RB1', 'CDKN1B', 'TP53', 'CDC16', 'ETS2', 'TERF2IP', 'EHMT1', 'ANAPC5', 'HIST1H2BM', 'CDK6', 'CBX4', 'LMNB1', 'UBE2E1', 'CDKN2C', 'MAPKAPK3', 'MAP2K6', 'MAP2K7', 'HIST1H2AD', 'CBX2', 'UBC', 'MOV10', 'HIST1H2BL', 'ANAPC11', 'E2F1', 'TINF2', 'HIST2H2BE', 'HIST1H2BD', 'H1F0', 'HIST1H2AB', 'CBX6', 'CDC26', 'CABIN1', 'BMI1', 'ATM', 'HIRA', 'RPS6KA3', 'IL6'

In [4]:
%%time
adata = sc.read_h5ad("../07_final_RNA_without_scvi.h5ad")
adata

CPU times: user 15.6 s, sys: 1min 12s, total: 1min 28s
Wall time: 1min 28s


AnnData object with n_obs × n_vars = 2305964 × 16115
    obs: 'age', 'donor_id', 'sex', 'region', 'cell_type', 'disease', 'consistent_cell_type', 'study', 'technology', 'cell_or_nuclei', 'barcode', 'sample_id', 'age_status', 'tech_plus_study', 'disease_binary', 'decade', 'age_group', '_scvi_batch', '_scvi_labels', 'leiden_scVI', 'scvi_cell_type', 'redo_leiden_0.5', 'UMAP1', 'UMAP2', 'v2_scvi_cell_type', 'final_cell_type'
    obsm: 'X_scVI', 'X_umap', '_scvi_extra_categorical_covs'
    layers: 'counts'

### Calculate 3 different senescence scores: 

- SenMayo gene set
- p16/p21
- R-HSA-2559583

In [5]:
%%time
sc.tl.score_genes(adata, SenMayo_list)
adata.obs = adata.obs.rename(columns = {'score': 'SenMayo_score'})

sc.tl.score_genes(adata, ["CDKN1A", "CDKN2A"])
adata.obs = adata.obs.rename(columns = {'score': 'p16_21_score'})

sc.tl.score_genes(adata, reactome_senescence_list)
adata.obs = adata.obs.rename(columns = {'score': 'reactome_score'})

       'IGFBP1', 'IL13', 'IL1A', 'IL2', 'MMP10', 'MMP12', 'MMP13', 'MMP3',
       'PECAM1', 'SERPINB4', 'SPX', 'VGF'],
      dtype='object')
       'H2AFX', 'MIR24-2', 'HIST1H1E', 'HIST1H2BC', 'HIST1H1A', 'HIST2H3A',
       'HIST1H2BA', 'HIST1H2BB', 'H3F3A', 'CXCL8', 'ERF', 'MRE11A', 'HIST1H1C',
       'HIST2H2AA3', 'H2AFZ', 'HIST1H2BM', 'HIST1H2AD', 'HIST1H2BL',
       'HIST2H2BE', 'HIST1H2BD', 'H1F0', 'HIST1H2AB', 'HIST1H2AC', 'HIST3H3',
       'HIST1H1B', 'HIST1H3A', 'HIST1H2BO', 'MIR24-1', 'HIST1H2BH', 'H2AFB1',
       'HIST1H1D', 'IFNB1', 'HIST3H2BB', 'HIST1H4A', 'HIST1H2BN'],
      dtype='object')
CPU times: user 4min 26s, sys: 2min 56s, total: 7min 22s
Wall time: 7min 22s


In [6]:
# save the adata scores
adata_metadata = adata.obs

In [7]:
#sc.pl.umap(adata, color = "SenMayo_score", cmap=sns.cubehelix_palette(dark=0, light=.9, as_cmap=True))

In [8]:
def perform_score_regression(adata_metadata, cell_type_key, score_key, cell_type):
    '''
    Perform the linear regression of a senescence score (specified by score key) against the several covariates, specified by
    covariates_list (e.g., [sex, age_group]) for a particular cell type. This function will obtain the mean senescence score 
    across all of the cells for that cell_type_key belonging to the same donor (donor_id).
    All of these values are expected to be present within the adata_metadata (adata.obs).
    '''

    data = adata_metadata.copy()
    data = data[data[cell_type_key] == cell_type]

    # extract the relevant covariates
    donor_level_metadata =  ( data[["donor_id", "sex", "tech_plus_study", 
                                    "age_group", "disease_binary"]].drop_duplicates().reset_index(drop=True) )

    # get the mean scores per donor for the senescence score, use observed=True to focus only on the donors present
    mean_senescence_scores = data.groupby("donor_id", observed=True)[score_key].mean().reset_index().rename(columns = {score_key : "score"})

    # merge donor level metadata with the mean senescence scores for that donor
    updated_df = donor_level_metadata.merge(mean_senescence_scores)
    # make sure the order of the covariates is as desired; the first is the baseline 
    updated_df['sex'] = pd.Categorical(updated_df['sex'], categories=['female', 'male'], ordered=True)
    updated_df['age_group'] = pd.Categorical(updated_df['age_group'], categories=['young', 'fetal', 'middle', 'old'], ordered=True)
    updated_df['disease_binary'] = pd.Categorical(updated_df['disease_binary'], categories=['N', 'Y'], ordered=True)

    # set up the linear regression model
    model = smf.ols("score ~ sex + age_group + disease_binary + tech_plus_study", data=updated_df)
    result = model.fit()
    
    return result

In [9]:
def extract_aging_and_disease_coefficients_and_pvalues(model_results):
    '''Extract the coefficients for aging and disease from the model'''

    # extract all coefficients and p-values
    coefficients = model_results.params
    pvalues = model_results.pvalues

    # get the results for the desired contrasts
    contrasts = ['age_group[T.old]', 'disease_binary[T.Y]']
    regression_results = pd.DataFrame({
        "contrast": contrasts,
        "coefficient": [coefficients.get(contrast, None) for contrast in contrasts],
        "p_val": [pvalues.get(contrast, None) for contrast in contrasts]
    })

    return regression_results

### Run an example for one cell type and one score

In [10]:
# perform regression
cell_type_key = "final_cell_type"

results = perform_score_regression(adata_metadata = adata_metadata, 
                         cell_type_key=cell_type_key, 
                         score_key = "p16_21_score",
                         cell_type = "Cardiomyocyte")

# extract the coefficents for aging and disease in a df

extract_aging_and_disease_coefficients_and_pvalues(results)

,contrast,coefficient,p_val
0,age_group[T.old],0.006511,0.142509
1,disease_binary[T.Y],0.003145,0.449163


### We will now iterate through all cell types and all scores

In [11]:
# list of all cell types
cell_types = adata.obs[cell_type_key].unique()

# list of all scores
score_types = ["SenMayo_score", "p16_21_score", "reactome_score"]

In [12]:
%%time 

# iterate through and append to growing list
senescence_score_df_list = list()

for cell_type_val in cell_types:
    for score_type_val in score_types:

        results = perform_score_regression(adata_metadata = adata_metadata, 
                         cell_type_key=cell_type_key, 
                         score_key = score_type_val,
                         cell_type = cell_type_val)
        
        results_df = extract_aging_and_disease_coefficients_and_pvalues(results)

        # add the cell type and score information to the df, then append it
        results_df['cell_type'] = cell_type_val
        results_df['score_type'] = score_type_val

        senescence_score_df_list.append(results_df)

CPU times: user 3.46 s, sys: 1.13 s, total: 4.59 s
Wall time: 4.59 s


In [13]:
# combine the lists together
senescence_score_df = pd.concat(senescence_score_df_list).reset_index(drop=True)

### Perform BH correction and add this to the dataframe

In [14]:
p_values = senescence_score_df['p_val']
adjusted_pvals = multipletests(p_values, method='fdr_bh')[1]
senescence_score_df['adjusted_p_val'] = adjusted_pvals

In [15]:
# visualize the significant p-values
significant_senescence_df = senescence_score_df[senescence_score_df['adjusted_p_val'] < 0.05].reset_index(drop=True)
significant_senescence_df.head()

,contrast,coefficient,p_val,cell_type,score_type,adjusted_p_val
0,disease_binary[T.Y],-0.009563,0.000118,Endothelial,SenMayo_score,0.004595
1,disease_binary[T.Y],-0.034663,0.000026,vSMC,p16_21_score,0.001990
2,disease_binary[T.Y],-0.009238,0.000290,Myeloid,reactome_score,0.007542


In [16]:
print(significant_senescence_df.shape)

(3, 6)


In [17]:
# filter to disease related, and sort by coefficient
significant_senescence_df[significant_senescence_df['contrast'] == "disease_binary[T.Y]"].sort_values(by = "coefficient")

,contrast,coefficient,p_val,cell_type,score_type,adjusted_p_val
1,disease_binary[T.Y],-0.034663,0.000026,vSMC,p16_21_score,0.001990
0,disease_binary[T.Y],-0.009563,0.000118,Endothelial,SenMayo_score,0.004595
2,disease_binary[T.Y],-0.009238,0.000290,Myeloid,reactome_score,0.007542


In [18]:
# filter to disease related, and sort by coefficient
significant_senescence_df[significant_senescence_df['contrast'] == "age_group[T.old]"].sort_values(by = "coefficient")

,contrast,coefficient,p_val,cell_type,score_type,adjusted_p_val


### Conclusion: not many significant disease or aging associations across cell types. 

- There are no significant association with aging after multiple hypothesis correction. Surprisingly, disease is associated with decreased senescence scores in most cell types.

In [19]:
# save as file for next script to produce ggplots
senescence_score_df.to_csv("01_senescence_results.csv")